# meeting_mcp Colab / ngrok helper

This notebook prepares a Colab environment to run the `meeting_mcp` FastAPI backend and Streamlit UI, exposes them via ngrok, and copies local model files from Google Drive into the repo for faster local loading.

Run cells in order. Update paths and tokens where prompted.

In [ ]:
# (Optional) Install Python dependencies. Uncomment to run in Colab if needed.
# NOTE: installing all requirements may take several minutes.
# Install from the repo requirements (adjust path if you cloned into /content)
# !pip install -r /content/AIDrivenMeetingSummaryProjectRiskDetection/requirements.txt --quiet
# Common useful packages (uncomment as needed):
# !pip install streamlit pyngrok --quiet
# !pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118 --quiet
# !pip install bitsandbytes --quiet
# !pip install uvicorn --quiet
print('Skip installs if your environment already has the required packages.')

In [ ]:
# Clone the repository into /content in Colab if not already present, and set REPO_ROOT
import os, subprocess, pathlib
repo_dir = '/content/AIDrivenMeetingSummaryProjectRiskDetection'
if not pathlib.Path(repo_dir).exists():
    print('Cloning repository to', repo_dir)
    try:
        subprocess.check_call(['git','clone','https://github.com/NaveenPalisetti/AIDrivenMeetingSummaryProjectRiskDetection.git', repo_dir])
    except Exception as e:
        print('git clone failed:', e)
else:
    print('Repository already exists at', repo_dir)
os.environ['REPO_ROOT'] = repo_dir
print('Set REPO_ROOT =', os.environ['REPO_ROOT'])
# Change working directory for the notebook runtime to the repo root
try:
    os.chdir(repo_dir)
    print('Changed CWD to', repo_dir)
except Exception as e:
    print('Failed to chdir to repo root:', e)

In [ ]:
# Install requirements from the cloned repository (uncomment to run).
import os
repo = os.environ.get('REPO_ROOT', '/content/AIDrivenMeetingSummaryProjectRiskDetection')
print('Using repo:', repo)
# Uncomment the following lines to install dependencies in Colab
# !pip install -r $repo/requirements.txt
# Install package in editable mode during development
# !pip install -e $repo
# Heavy / GPU packages (uncomment if you need GPU support):
# !pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118 --quiet
# !pip install bitsandbytes --quiet
# !pip install streamlit pyngrok uvicorn --quiet

In [ ]:
# Install minimal dependencies and optionally copy Mistral model from Drive to /content for faster loading.
# Uncomment the pip line when running in Colab (it may take several minutes).
# !pip install -r /content/AIDrivenMeetingSummaryProjectRiskDetection/requirements.txt --quiet
import shutil, os, pathlib
# Optional: copy Mistral model from Google Drive to local disk (adjust source path if needed)
src = '/content/drive/MyDrive/Dissertation/Project/models/mistral-7B-Instruct-v0.2'
dst = '/content/mistral-7B-Instruct-v0.2'
if os.path.exists(src):
    if not os.path.exists(dst):
        print('Copying Mistral model from Drive to local disk...')
        try:
            shutil.copytree(src, dst)
            print('Copied Mistral model to', dst)
        except Exception as e:
            print('Failed to copy Mistral model:', e)
    else:
        print('Local copy of Mistral model already exists at', dst)
else:
    print('Source Mistral model not found at', src, '— ensure Drive is mounted and path is correct')

In [ ]:
# Mount Google Drive (only required in Colab) and set model paths.
import os
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print('Mounted Google Drive at /content/drive')
except Exception:
    print('google.colab not available — ensure models are accessible locally (not in Drive).')

# Default Drive paths used previously in this project — update if your Drive uses different paths
os.environ.setdefault('BART_MODEL_PATH', '/content/drive/MyDrive/Dissertation/Project/models/local_bart_large_cnn_finetuned')
os.environ.setdefault('MISTRAL_MODEL_PATH', '/content/drive/MyDrive/Dissertation/Project/models/mistral-7B-Instruct-v0.2')
print('BART_MODEL_PATH =', os.environ.get('BART_MODEL_PATH'))
print('MISTRAL_MODEL_PATH =', os.environ.get('MISTRAL_MODEL_PATH'))

In [ ]:
# Install requirements from the repo if you prefer the workspace copy (uncomment to run).
# Adjust the path to your repo location if different.
# !pip install -r /content/drive/MyDrive/Dissertation/Project/requirements.txt --quiet
print('If you need dependencies installed, uncomment the pip command in this cell and run it.')

In [ ]:
# Copy model directories from Drive into local ./models/ for faster local loading.
import os, shutil, pathlib
dst_root = pathlib.Path('models')
dst_root.mkdir(exist_ok=True)
# Copy BART model if present
bart_src = os.environ.get('BART_MODEL_PATH')
bart_dst = dst_root / 'local_bart_large_cnn_finetuned'
if bart_src and os.path.exists(bart_src):
    try:
        if bart_dst.exists():
            print(f'BART destination {bart_dst} exists — skipping copy')
        else:
            print(f'Copying BART model from {bart_src} to {bart_dst} ...')
            shutil.copytree(bart_src, bart_dst)
            os.environ['BART_MODEL_PATH'] = str(bart_dst)
            print('Copied BART model and updated BART_MODEL_PATH')
    except Exception as e:
        print('Failed to copy BART model:', e)
else:
    print('BART_MODEL_PATH not set or source missing — skipping BART copy')

# Copy Mistral model if present
mistral_src = os.environ.get('MISTRAL_MODEL_PATH')
mistral_dst = dst_root / 'mistral-7B-Instruct-v0.2'
if mistral_src and os.path.exists(mistral_src):
    try:
        if mistral_dst.exists():
            print(f'Mistral destination {mistral_dst} exists — skipping copy')
        else:
            print(f'Copying Mistral model from {mistral_src} to {mistral_dst} ...')
            shutil.copytree(mistral_src, mistral_dst)
            os.environ['MISTRAL_MODEL_PATH'] = str(mistral_dst)
            print('Copied Mistral model and updated MISTRAL_MODEL_PATH')
    except Exception as e:
        print('Failed to copy Mistral model:', e)
else:
    print('MISTRAL_MODEL_PATH not set or source missing — skipping Mistral copy')

In [ ]:
# Prepare ngrok auth token and MCP_API_KEY (optional).
import os
if not os.environ.get('NGROK_AUTH_TOKEN'):
    try:
        v = input('Enter NGROK_AUTH_TOKEN (leave blank to skip): ').strip()
        if v:
            os.environ['NGROK_AUTH_TOKEN'] = v
    except Exception:
        pass
if not os.environ.get('MCP_API_KEY'):
    try:
        v = input('Enter MCP_API_KEY to protect the HTTP API (optional): ').strip()
        if v:
            os.environ['MCP_API_KEY'] = v
    except Exception:
        pass
print('NGROK_AUTH_TOKEN set:', bool(os.environ.get('NGROK_AUTH_TOKEN')) )
print('MCP_API_KEY set:', bool(os.environ.get('MCP_API_KEY')) )

In [ ]:
# Start FastAPI (uvicorn) in background and capture PID.
import subprocess, time, os
from pathlib import Path
# Prefer an explicit REPO_ROOT or GITHUB_WORKSPACE if set (useful after git clone)
repo_root = Path(os.getenv('REPO_ROOT') or os.getenv('GITHUB_WORKSPACE') or Path.cwd())
print('Using repo_root =', repo_root)
try:
    os.chdir(repo_root)
except Exception as e:
    print('Failed to chdir to repo_root:', e)
UVICORN_CMD = ['uvicorn','meeting_mcp.server.mcp_api:app','--host','127.0.0.1','--port','8000','--reload']
logf = open('uvicorn_meeting_mcp.log','a')
try:
    proc = subprocess.Popen(UVICORN_CMD, stdout=logf, stderr=logf)
    print('uvicorn started, pid=', proc.pid)
    (Path('uvicorn_meeting_mcp.pid')).write_text(str(proc.pid))
except Exception as e:
    print('Failed to start uvicorn:', e)
time.sleep(3)
print('Waiting for FastAPI to warm up...')
time.sleep(5)

In [ ]:
# Start Streamlit UI in background and create ngrok tunnels for backend and UI.
from pyngrok import ngrok, conf
import os, subprocess, time
from pathlib import Path
if os.environ.get('NGROK_AUTH_TOKEN'):
    conf.get_default().auth_token = os.environ.get('NGROK_AUTH_TOKEN')
# Start Streamlit app (use the meeting_mcp Streamlit client)
ST_CMD = ['streamlit','run','meeting_mcp/ui/streamlit_agent_client.py','--server.port','8501']
try:
    st_proc = subprocess.Popen(ST_CMD, stdout=open('streamlit_meeting_mcp.log','a'), stderr=open('streamlit_meeting_mcp.err','a'))
    print('Streamlit started, pid=', st_proc.pid)
    (Path('streamlit_meeting_mcp.pid')).write_text(str(st_proc.pid))
except Exception as e:
    print('Failed to start Streamlit:', e)
time.sleep(3)
# Kill existing tunnels to avoid duplicates
try:
    ngrok.kill()
except Exception:
    pass
print('Opening ngrok tunnel to backend (8000) ...')
backend_tunnel = ngrok.connect(8000, bind_tls=True)
backend_url = backend_tunnel.public_url
print('Backend public URL:', backend_url)
os.environ['ORCHESTRATOR_BASE'] = backend_url
os.environ['ORCHESTRATOR_URL'] = backend_url + '/mcp/orchestrate'
time.sleep(1)
print('Opening ngrok tunnel to Streamlit UI (8501) ...')
ui_tunnel = ngrok.connect(8501, bind_tls=True)
ui_url = ui_tunnel.public_url
print('Streamlit public URL:', ui_url)
print('Set ORCHESTRATOR_URL in your Streamlit client or use the UI URL above to access the app.')

In [ ]:
# Smoke test the orchestrator endpoint to ensure the backend responds.
import requests, json, os
backend = os.environ.get('ORCHESTRATOR_BASE') or 'http://127.0.0.1:8000'
test_url = backend + '/mcp/orchestrate'
payload = {'message':'ping','params':{}}
try:
    r = requests.post(test_url, json=payload, timeout=15)
    print('Status code:', r.status_code)
    try:
        print('Response:', json.dumps(r.json(), indent=2))
    except Exception:
        print('Non-JSON response:', r.text[:1000])
except Exception as e:
    print('Failed to call orchestrator endpoint:', e)

## Troubleshooting & next steps
- If endpoints fail, check `uvicorn_meeting_mcp.log` and `streamlit_meeting_mcp.log` for errors.
- To stop background services, run `ngrok.kill()` and kill PIDs recorded in `uvicorn_meeting_mcp.pid` and `streamlit_meeting_mcp.pid`.
- If model loading fails, verify `BART_MODEL_PATH` and `MISTRAL_MODEL_PATH` point to valid local folders containing model files (config.json, pytorch_model.bin or similar).